In [1]:
from IPython import get_ipython
from ipywidgets import *

import matplotlib.pyplot as plt

import zarr
import s3fs
import dask.array as da
import requests

In [2]:
ENDPOINT_URL = "https://cellpainting-gallery.s3.amazonaws.com/"
ROOT = "cpg0004-lincs/broad/images/2016_04_01_a549_48hr_batch1/images_zarr/SQ00015098__2016-06-08T18_43_42-Measurement1.ome.zarr"

## Read information about the plate 

In [3]:
zattrs = requests.get(ENDPOINT_URL + ROOT + "/.zattrs").json()
print(zattrs)

{'bioformats2raw.layout': 3, 'plate': {'columns': [{'name': '1'}, {'name': '2'}, {'name': '3'}, {'name': '4'}, {'name': '5'}, {'name': '6'}, {'name': '7'}, {'name': '8'}, {'name': '9'}, {'name': '10'}, {'name': '11'}, {'name': '12'}, {'name': '13'}, {'name': '14'}, {'name': '15'}, {'name': '16'}, {'name': '17'}, {'name': '18'}, {'name': '19'}, {'name': '20'}, {'name': '21'}, {'name': '22'}, {'name': '23'}, {'name': '24'}], 'name': 'SQ00015098', 'wells': [{'path': 'A/1', 'rowIndex': 0, 'columnIndex': 0}, {'path': 'A/2', 'rowIndex': 0, 'columnIndex': 1}, {'path': 'A/3', 'rowIndex': 0, 'columnIndex': 2}, {'path': 'A/4', 'rowIndex': 0, 'columnIndex': 3}, {'path': 'A/5', 'rowIndex': 0, 'columnIndex': 4}, {'path': 'A/6', 'rowIndex': 0, 'columnIndex': 5}, {'path': 'A/7', 'rowIndex': 0, 'columnIndex': 6}, {'path': 'A/8', 'rowIndex': 0, 'columnIndex': 7}, {'path': 'A/9', 'rowIndex': 0, 'columnIndex': 8}, {'path': 'A/10', 'rowIndex': 0, 'columnIndex': 9}, {'path': 'A/11', 'rowIndex': 0, 'columnI

In [4]:
print((zattrs['plate']['rows'][0]['name']))

A


In [6]:
from ipysheet import sheet, cell

sheet1 = sheet(rows=len(zattrs['plate']['rows']), columns=len(zattrs['plate']['columns']))
for well in zattrs['plate']['wells']:
    c = cell(well['rowIndex'], well['columnIndex'], well['path'] + "/" + str(well['rowIndex']) + "/" + str(well['columnIndex']))

display(sheet1)

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='A/1/0/0'), Cell(co…

In [39]:
def load_dask_array_from_s3(row='0', column='0', name='A/1'):
    cache_size_mb = 2048
    cfg = {
        'anon': True,
        'client_kwargs': {
            'endpoint_url': 'https://cellpainting-gallery.s3.amazonaws.com/',
        },
        'root': 'cpg0004-lincs/broad/images/2016_04_01_a549_48hr_batch1/images_zarr/SQ00015098__2016-06-08T18_43_42-Measurement1.ome.zarr/%s/%s/%s/' % (name, row, column)
    }
    s3 = s3fs.S3FileSystem(
        anon=cfg['anon'],
        client_kwargs=cfg['client_kwargs'],
    )
    print(cfg['root'])
    store = s3fs.S3Map(root=cfg['root'], s3=s3, check=False)
    cached_store = zarr.LRUStoreCache(store, max_size=(cache_size_mb * 2**20))
    # data.shape is (t, c, z, y, x) by convention
    return da.from_zarr(cached_store)

In [40]:
%time data = load_dask_array_from_s3()

cpg0004-lincs/broad/images/2016_04_01_a549_48hr_batch1/images_zarr/SQ00015098__2016-06-08T18_43_42-Measurement1.ome.zarr/A/1/0/0/
CPU times: user 29.5 ms, sys: 13.8 ms, total: 43.3 ms
Wall time: 557 ms


In [41]:
get_ipython().run_line_magic('matplotlib', 'inline')

from skimage import filters
from skimage.morphology import closing, square
from skimage import measure

def display_image(channel=0, v=5):
    img = data[0,channel,0,:,:]
    # denoising
    blurred = filters.median(img)
    # binarization
    threshold = filters.threshold_otsu(blurred)
    binary = closing(blurred <= threshold, square(5))
    label_image = measure.label(binary, background=1)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(img)
    ax.contour(label_image, [0.5], linewidths=1.2, colors='r')
    plt.show()
    return label_image


output = interactive(display_image, data=data,
                     channel=widgets.IntSlider(value=0, min=0, max=data.shape[1], step=1, description="Channel", continuous_update=False),
                     v=widgets.IntSlider(value=5, min=1, max=10, step=1, description="Threshold", continuous_update=False))
display(output)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Channel', max=5), IntSlider(val…